## Ejercicio: Titanic

1. Carga: toma como base el fichero titanic_train.csv y cárgalo.
2. Visual EDA:
   ¿Exiset algún patrón de supervivencia basado en el sexo?
   Crea el siguiente gráfico: sns.countplot(x='Survived',hue='Sex',data=train,palette='RdBu_r')
    ¿Y basado en el clase de los pasajeros (pclass)?
    Crea un histograma que represente la edad de los pasajeros.
    Contrasta la clase respecto a la edad.
3. Preprocessing:
    Sustituye los missing values de la edad por el valor medio.
    Suprime columnas innecesarias.
    Convierte columnas categóricas por variables dummies
4. Creación modelo de regresión logística
5. Entrenamiento
6. Obtén el informe de clasificación y la matriz de confusión.


1. Carga: toma como base el fichero titanic_train.csv y cárgalo.## 1. Importación de paquetes y dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
names = 'response age sex on_thyroxine query_on_thyroxine antithyroid_medication thyroid_surgery query_hypothyroid query_hyperthyroid pregnant \
sick tumor lithium goitre TSH_measured TSH T3_measured \
T3 TT4_measured TT4 T4U_measured T4U FTI_measured FTI TBG_measured TBG'

In [ ]:
names = names.split(' ')

In [ ]:
df = pd.read_csv('hypothyroid.csv',index_col=False,names=names,na_values=['?'])

In [ ]:
df.head()

In [ ]:
df.shape

### 3. Preprocessing:Sustituye los missing values de la edad por el valor medio

3. Preprocessing:


    Sustituye los missing values de la edad por el valor medio.


    Suprime columnas innecesarias.


    Convierte columnas categóricas por variables dummies


5. Entrenamiento


6. Obtén el informe de clasificación y la matriz de confusión.

4. Creación modelo de regresión logística


In [ ]:
to_drop=[]
for c in df.columns:
    if 'measured' in c or 'query' in c:
        to_drop.append(c)

In [ ]:
to_drop

In [ ]:
to_drop.append('TBG')

In [ ]:
df.drop(to_drop,axis=1,inplace=True)

In [ ]:
df.head()

Consultamos los valores estadísticos básicos de cada columna. Si añadimos .T a la función 'describe' podemos transponer la matriz resultado y ayuda a la comprensión de dichos valores.

In [ ]:
df.describe().T

### 2.2. Tratamiento de missing values
La función `df.isna()` retorna un dataframe de valores booleanos (true para indicar la existencia de valor y false para valores missing). Si añadimos la función sum se verán y calcularán el número de missing values para cada columna.

In [ ]:
df.isna().sum()

En este caso borraremos las filas con valores missing ya que desconocemos de conocimiento experto en el área que nos ayude a estimar correctamente dichos valores desconocidos.

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape
df['response'].unique()

### 2.3. Transformación de datos categóricos manualmente

La columna **response** contiene los valores 'hypothyroid' para indicar que el paciente sufre hipertiroidismo y 'negative' para indicar que no. Los sustituiremos manualmente por valores numéricos (hypothyroid=1 y negative=0).

In [ ]:
def class_convert(response):
    if response=='hypothyroid':
        return 1
    else:
        return 0

In [ ]:
df['response']=df['response'].apply(class_convert)

In [ ]:
df.head()



In [ ]:
df.columns

## 3. Visual EDA

### 3.1. Diagrama de cajas

Un diagrama de caja (en inglés boxplot) es un método estandarizado para representar gráficamente una serie de datos numéricos a través de sus cuartiles. De esta manera, se muestran a simple vista la mediana(el valor que ocupa la posición central de un conjunto de observaciones ordenadas) y los cuartiles de los datos, y también pueden representarse sus valores atípicos (o outlayers).
![Explicacion boxplot](./data/box_plot.png) 

In [ ]:
for var in ['age','TSH','T3','TT4','T4U','FTI']:
    sns.boxplot(x='response',y=var,data=df)
    plt.show()

### 3.2. Análisis de correlaciones entre variables independendientes

In [ ]:
sns.pairplot(data=df[df.columns[1:]],diag_kws={'edgecolor':'k','bins':25},plot_kws={'edgecolor':'k'})
plt.show()

### 3.3. Transformación de datos categóricos con get_dummies

In [ ]:
df_dummies = pd.get_dummies(data=df)

In [ ]:
df_dummies.shape

In [ ]:
df_dummies.sample(10)

## 4. División del dataset

La variable X contendrá todas las columnas a excepción de response.
La variable y contendrá exclusivamente la columna response.

In [ ]:
from sklearn.model_selection import train_test_split
X = df_dummies.drop('response',axis=1)
y = df_dummies['response']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
print("Training set shape",X_train.shape)
print("Test set shape",X_test.shape)

## 5. Creación del modelo de Regresión Logística

Se implementará la regresión logística con regularización Ridge L2(por defecto). [Aquí](https://www.iartificial.net/regularizacion-lasso-l1-ridge-l2-y-elasticnet/) más información sobre los tipos de regularizaciones.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf1 = LogisticRegression(penalty='l2')

## 6. Entrenamiento

In [ ]:
clf1.fit(X_train,y_train)

## 7. Cálculo de intercept, coefficients y score

In [ ]:
clf1.intercept_

In [ ]:
clf1.coef_

In [ ]:
clf1.score(X_test,y_test)

## 8. Predicciones

Para las regresiones logísticas, el método `predict_proba` estima la probabilidad de ser de tipo 0 (NO) y la probabilidad de ser de tipo 1(YES).

In [ ]:
prob_threshold = 0.5

In [ ]:
prob_df=pd.DataFrame(clf1.predict_proba(X_test[:10]),columns=['Prob of NO','Prob of YES'])
prob_df['Decision']=(prob_df['Prob of YES']>prob_threshold).apply(int)
prob_df

In [ ]:
y_test[:10]

## 9. Informe de classificación (classification report) y matriz de confusión (confusion matrix)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(confusion_matrix(y_test, clf1.predict(X_test)))

In [ ]:
print(classification_report(y_test, clf1.predict(X_test)))

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix

# Cargar conjuntos de datos
df_train = pd.read_csv("titanic_train.csv")
df_test = pd.read_csv("titanic_test.csv")

# Visual EDA: Patrón de supervivencia basado en el sexo
sns.countplot(x='Survived', hue='Sex', data=df_train, palette='RdBu_r')
plt.title('Survival Count by Sex')
plt.show()

# Visual EDA: Patrón de supervivencia basado en la clase de los pasajeros (pclass)
sns.histplot(data=df_train, x='Age', hue='Pclass', bins=30, kde=True, palette='viridis')
plt.title('Age Distribution by Passenger Class')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

# Preprocesamiento
def preprocess_data(df):
    # Sustituir missing values de la edad por el valor medio
    df['Age'].fillna(df['Age'].mean(), inplace=True)

    # Suprimir columnas innecesarias
    columns_to_drop = ['Cabin', 'Ticket', 'Name', 'PassengerId']
    df = df.drop(columns=columns_to_drop)

    # Convertir columnas categóricas en variables dummy
    df = pd.get_dummies(df, columns=['Sex', 'Embarked'], drop_first=True)
    
    return df

df_train = preprocess_data(df_train)
df_test = preprocess_data(df_test)

# Creación del modelo de regresión logística
X = df_train.drop('Survived', axis=1)
y = df_train['Survived']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el preprocesamiento como un pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Age', 'Pclass', 'SibSp', 'Parch', 'Fare']),
        ('cat', 'passthrough', ['Sex_male', 'Embarked_Q', 'Embarked_S'])
    ]
)

# Crear y entrenar el modelo
model = make_pipeline(preprocessor, LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Obtener el informe de clasificación y la matriz de confusión en el conjunto de prueba
y_pred_test = model.predict(X_test)

# Mostrar resultados
print("Classification Report:")
print(classification_report(y_test, y_pred_test))

conf_matrix = confusion_matrix(y_test, y_pred_test)
print("\nConfusion Matrix:")
print(conf_matrix)

# Razonamientos interpretativos
precision_global = (conf_matrix[0, 0] + conf_matrix[1, 1]) / len(y_test) * 100
recall_0 = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
recall_1 = conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[1, 0])
f1_score_0 = 2 * (conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])) * recall_0
f1_score_1 = 2 * (conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[1, 0])) * recall_1

interpretation_str = f'''
Razonamientos:
- El modelo tiene una precisión global del {precision_global:.2f}%, indicando que el {precision_global:.2f}% de las predicciones en el conjunto de prueba son correctas.
- La precisión y recall son equilibrados, especialmente para la clase 0 (No sobrevivieron).
- La clase 1 (Sobrevivió) tiene un recall ligeramente más bajo ({recall_1:.2f}), indicando que el modelo podría perder algunos casos de sobrevivientes.
- F1-score proporciona un equilibrio entre precisión y recall, siendo {f1_score_0:.2f} para la clase 0 y {f1_score_1:.2f} para la clase 1.
- En la matriz de confusión, los True Positives (TP) y True Negatives (TN) son {tuple(conf_matrix[0])}, indicando una buena capacidad del modelo para clasificar ambas clases.
- Los False Positives (FP) y False Negatives (FN) son {tuple(conf_matrix[1])}, pero no despreciables. Se podrían ajustar los umbrales según las necesidades específicas del problema.
'''

print("\nRazonamientos Interpretativos:")
print(interpretation_str)


ModuleNotFoundError: No module named 'seaborn'

## Ejercicio: Titanic

1. Carga: toma como base el fichero titanic_train.csv y cárgalo.
2. Visual EDA:
   ¿Exiset algún patrón de supervivencia basado en el sexo?
   Crea el siguiente gráfico: sns.countplot(x='Survived',hue='Sex',data=train,palette='RdBu_r')
    ¿Y basado en el clase de los pasajeros (pclass)?
    Crea un histograma que represente la edad de los pasajeros.
    Contrasta la clase respecto a la edad.
3. Preprocessing:
    Sustituye los missing values de la edad por el valor medio.
    Suprime columnas innecesarias.
    Convierte columnas categóricas por variables dummies
4. Creación modelo de regresión logística
5. Entrenamiento
6. Obtén el informe de clasificación y la matriz de confusión.
